In [1]:
import sys
sys.path.append('../')
from read_data_functions import *
from plot_functions import *
from calculations import *
plt.style.use('../Style.mplstyle')
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress warnings

In [ ]:
path = '../../../Data/2025/250929-251003_Ulva/PTR-MS'
data = import_PTRMS(path, '')

dict_keys = []
for key in data.keys():
    if 'Traces' in key:
        if 'Conc' in key:
            dict_keys.append(key)

concentrations = ['002', '07', '39', '19']
conc_labels = [r'0.02 $\mu$M', r'0.7 $\mu$M', r'3.9 $\mu$M', r'19 $\mu$M']

print(dict_keys)
display(data['Traces_P002_Conc'])
print(data['Traces_P002_Conc'].keys())
print(data['Traces_P07_Conc'].keys())
print(data['Traces_P39_Conc'].keys())
print(data['Traces_P19_Conc'].keys())

timestamps = [[['2025-09-30 12:54', '2025-09-30 14:33'],
               ['2025-10-02 17:06', '2025-10-02 18:52']],   # [P] = 0.02 uM
              [['2025-10-01 16:02', '2025-10-01 17:44'],
               ['2025-10-03 10:15', '2025-10-03 11:58']],   # [P] = 0.7 uM
              [['2025-09-29 17:25', '2025-09-29 19:06'],
               ['2025-10-01 09:47', '2025-10-01 11:26'],
               ['2025-10-02 13:38', '2025-10-02 15:22']],   # [P] = 3.9 uM
              [['2025-09-30 16:29', '2025-09-30 18:09'],     
               ['2025-10-02 10:06', '2025-10-02 12:01'],
               ['2025-10-03 13:32', '2025-10-03 15:12']]]   # [P] = 19 uM


['Traces_P002_Conc', 'Traces_P07_Conc', 'Traces_P19_Conc', 'Traces_P39_Conc']


,AbsTime,RelTime,Cycle,CycleInFile,m63.026 (C[12]2H[1]7S[32]1) (Conc),m105.930 (C[12]1N[14]1Br[79]1H[1]1) (Conc),m153.915 (C[12]1N[14]1I[127]1H[1]1) (Conc),m266.813 (C[12]1I[127]2H[1]1) (Conc),Filename,Time
0,45929.366005,0.000000,1.0,1.0,0.007164,0.020165,0.000449,0.000074,P002_R1.h5,2025-09-29 08:47:02.799600051
1,45929.366062,4.999832,2.0,2.0,0.006004,0.016892,0.000483,0.000281,P002_R1.h5,2025-09-29 08:47:07.799481781
2,45929.366120,9.999663,3.0,3.0,0.004312,0.013444,0.000186,0.000531,P002_R1.h5,2025-09-29 08:47:12.799276756
3,45929.366178,14.999496,4.0,4.0,0.004686,0.017370,0.000313,0.000595,P002_R1.h5,2025-09-29 08:47:17.799158486
4,45929.366236,19.999327,5.0,5.0,0.003511,0.016219,0.000576,0.000370,P002_R1.h5,2025-09-29 08:47:22.798953453
...,...,...,...,...,...,...,...,...,...,...
6751,45932.792862,296080.501174,6752.0,2316.0,0.002722,0.018426,0.001319,0.003043,P002_R3.h5,2025-10-02 19:01:43.300819355
6752,45932.792920,296085.501007,6753.0,2317.0,0.005541,0.017850,0.000985,0.000901,P002_R3.h5,2025-10-02 19:01:48.300614330
6753,45932.792978,296090.500839,6754.0,2318.0,0.004614,0.018202,0.000370,0.001326,P002_R3.h5,2025-10-02 19:01:53.300496060
6754,45932.793036,296095.500671,6755.0,2319.0,0.003681,0.015976,0.000836,0.002140,P002_R3.h5,2025-10-02 19:01:58.300291035


Index(['AbsTime', 'RelTime', 'Cycle', 'CycleInFile',
       'm63.026 (C[12]2H[1]7S[32]1) (Conc)',
       'm105.930 (C[12]1N[14]1Br[79]1H[1]1) (Conc)',
       'm153.915 (C[12]1N[14]1I[127]1H[1]1) (Conc)',
       'm266.813 (C[12]1I[127]2H[1]1) (Conc)', 'Filename', 'Time'],
      dtype='object')
Index(['AbsTime', 'RelTime', 'Cycle', 'CycleInFile',
       'm63.026 (C[12]2H[1]7S[32]1) (Conc)', 'm81.070 (C[12]6H[1]9) (Conc)',
       'm95.085 (C[12]7H[1]11) (Conc)', 'm123.117 (C[12]9H[1]15) (Conc)',
       'm135.116 (C[12]10H[1]15) (Conc)', 'm137.132 (C[12]10H[1]17) (Conc)',
       'm151.147 (C[12]11H[1]19) (Conc)',
       'm153.914 (C[12]1N[14]1I[127]1H[1]1) (Conc)',
       'm155.062 (C[12]10H[1]7N[14]2) (Conc)',
       'm157.078 (C[12]9H[1]14Cl[35]1) (Conc)',
       'm171.094 (C[12]10H[1]16Cl[35]1) (Conc)',
       'm175.107 (C[12]12H[1]15O[16]1) (Conc)', 'Filename', 'Time'],
      dtype='object')
Index(['AbsTime', 'RelTime', 'Cycle', 'CycleInFile',
       'm63.026 (C[12]2H[1]7S[32]1) (Conc)

In [3]:
for key in dict_keys:
    for i, conc in enumerate(concentrations):
        if conc in key:
            print(key)
            conc_mean, conc_std = exp_mean(data[key], data[key].keys()[4:-2], timestamps[i])
            print(conc_mean)


Traces_P002_Conc
[0.15384521 0.0955916  0.02790171 0.0120891 ]
Traces_P07_Conc
[0.42468948 0.30902096 0.2407029  0.57109901 0.06409706 0.49804127
 0.15364466 0.01754268 0.02386457 0.20664514 0.1524449  0.02534809]
Traces_P19_Conc
[0.32379924 0.32963328 0.27939564 0.08312027 0.52692709 0.0781251
 0.59996286 0.1984928  0.02884557 0.02469021 0.22502407 0.17742909
 0.03107078 0.04639728 0.02870907 0.01249131]
Traces_P39_Conc
[0.35319738 0.16936158 0.22592166 0.05214144 0.27428663 0.11055853
 0.02924427 0.02379718 0.05858987]
